### Q1 Querying Users

In [2]:
import pyspark as spark
from pyspark import SparkContext, SparkConf
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import split, col, udf, sum

24/10/28 15:52:16 WARN Utils: Your hostname, macbook.local resolves to a loopback address: 127.0.0.1; using 10.243.54.6 instead (on interface en0)
24/10/28 15:52:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/28 15:52:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
df = spark.createDataFrame([("1",), ("2",)], ["col1"])

# Cast the column to string type
# df = df.withColumn("col1_str", col("col1").cast("string"))

# Show the DataFrame
df.show()
# print(type(df.tail(1)[0]))
print(df.tail(1)[0]['col1'])

+----+
|col1|
+----+
|   1|
|   2|
+----+

2


In [11]:
# queries contains about 300 data points, or roughly 5% of the total queries allowed
queries = spark.read.text("queries.txt")
queries = queries.select(split(queries.value," ")).rdd.flatMap(lambda x: x).toDF(schema = ['a','b','c','d','user','e','f','query'])
# drop useless columns.
queries = queries.drop('a','b','c','d','e','f')
queries.show()

+--------+-------+
|    user|  query|
+--------+-------+
|sndr0000|qry0045|
|sndr0000|qry0049|
|sndr0001|qry0048|
|sndr0000|qry0055|
|sndr0004|qry0014|
|sndr0003|qry0038|
|sndr0005|qry0024|
|sndr0002|qry0039|
|sndr0005|qry0011|
|sndr0001|qry0041|
|sndr0001|qry0001|
|sndr0000|qry0040|
|sndr0000|qry0035|
|sndr0005|qry0012|
|sndr0000|qry0040|
|sndr0002|qry0035|
|sndr0004|qry0029|
|sndr0004|qry0043|
|sndr0004|qry0034|
|sndr0001|qry0026|
+--------+-------+
only showing top 20 rows



### finding number of duplicates for each user

In [15]:
users = ['sndr0000','sndr0001','sndr0002','sndr0003','sndr0004','sndr0005']
for u in users:
    print(u,'duplicate queries')
    s1 = queries.where(col("user") == u)
    s1 = s1.groupBy(s1.columns)\
        .count()\
        .where(col('count') > 1)\
        .select(F.sum('count'))\
        .show()


sndr0000 duplicate queries
+----------+
|sum(count)|
+----------+
|        25|
+----------+

sndr0001 duplicate queries
+----------+
|sum(count)|
+----------+
|        21|
+----------+

sndr0002 duplicate queries
+----------+
|sum(count)|
+----------+
|        23|
+----------+

sndr0003 duplicate queries
CodeCache: size=131072Kb used=30515Kb max_used=30526Kb free=100557Kb
 bounds [0x00000001069f8000, 0x00000001087f8000, 0x000000010e9f8000]
 total_blobs=11365 nmethods=10412 adapters=864
 compilation: disabled (not enough contiguous free space left)


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


+----------+
|sum(count)|
+----------+
|        33|
+----------+

sndr0004 duplicate queries
+----------+
|sum(count)|
+----------+
|        30|
+----------+

sndr0005 duplicate queries
+----------+
|sum(count)|
+----------+
|        29|
+----------+



### calculating average number of duplicates by any user

In [16]:
print((25+21+23+29+30+33)/6)

26.833333333333332


## Q2 Bloom Filter

### implementing the bloom filter

In [35]:
from pyspark.sql.types import StringType
import base64
import math
import mmh3
from bitarray import bitarray

class filter(object):

    def __init__(self, items_count, size, fil):
        
        # Size of bit array to use
        self.size = size
        # number of hash functions to use
        self.hash_count = self.get_hash_num(self.size, items_count)

        # Bit array of given size
        self.bit_array = fil

    # check if item is in filter
    def check(self, item):
        for i in range(self.hash_count):
            digest = mmh3.hash(item, i) % self.size
            # if false bit detected, return false
            if self.bit_array[digest] == False:
                return False
        return True

    # return the number of hash functions needed. Number is calculated based on size of the array and number of input words
    def get_hash_num(self, s, n):
        k = (s/n) * math.log(2)
        return int(k)

def filter_negativity(word):
    
    with open('vector.txt', 'r') as file:
        file_content = file.read()
        file_content = base64.b64decode(file_content)

    fil = bitarray(file_content.decode("utf-8"))
    bloom = filter(63, 1024, fil)
    return bloom.check(word)
# print(filter_negativity('happy'))
# print(filter_negativity('damned'))
negUDF = udf(lambda x:filter_negativity(x),StringType()) 

False
True


In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# apply filter to each word
negativity = words.select(col("word"), negUDF(col("word")).alias("nagative") ).show()
                          
'''# Custom UDF with select()  
df.select(col("Seqno"), upperCaseUDF(col("Name")).alias("Name") ).show(truncate=False)'''

 # Start running the query that prints the words and their AFINN negativity aaffiliation to the console
query = negativity \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

## Q3 Hyperloglog

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



## References
[1] https://www.statology.org/pyspark-count-duplicate-rows/

[2] https://stackoverflow.com/questions/48554619/count-number-of-duplicate-rows-in-sparksql

[3] https://www.statology.org/pyspark-count-duplicate-rows/

[4] https://www.geeksforgeeks.org/bloom-filters-introduction-and-python-implementation/